In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from qgsw.utils.sorting import sort_files
from mpl_toolkits.axes_grid1 import make_axes_locatable
from qgsw.run_summary import RunSummary
from qgsw.models.qg.alpha import extract_coefficients_from_run

In [ ]:
ROOT = Path(os.path.abspath('')).parent

In [ ]:
folder = ROOT.joinpath("output/g5k/imports")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))

In [ ]:
from qgsw.models.qg.alpha import save_coefficients

times, coefs = extract_coefficients_from_run(folder)
save_coefficients(ROOT.joinpath("data/coefficients.npz"),times, coefs)

In [ ]:
_, files = sort_files(folder.rglob("*.npz"), prefix=run.configuration.model.prefix, suffix = ".npz")

In [ ]:
fields_map = {
    "vorticity": {"key": "omega", "label":"Vorticity"},
    "pressure": {"key": "p", "label":"Pressure"},
}
field = fields_map["vorticity"]

In [ ]:
# %matplotlib inline
plot = True
if plot:
    plt.ion()
    cmap = plt.cm.bwr
    fig, axes = plt.subplots(2, 2, figsize=(12,12))
    fig.subplots_adjust(right=0.85)

    cbars = np.empty_like(axes)

    cbars[0,0] = make_axes_locatable(axes[0,0]).append_axes('right', size='5%', pad=0.05)
    cbars[0,1] = make_axes_locatable(axes[0,1]).append_axes('right', size='5%', pad=0.05)

offset = 0
last = -1
for i in range(len(files[offset:last])+1):
    file = files[i+offset]
    time = times[i+offset]
    data = np.load(file)[field["key"]]
    top = data[0,0]
    bottom = data[0,1]

    if plot:
        fig.suptitle(f"Step: {time} / {times[last]}")

        axes[0,0].cla()
        axes[0,1].cla()
        axes[1,0].cla()

        cbar = axes[0,0].imshow(top, cmap=cmap, vmin=-np.max(np.abs(top)), vmax=np.max(np.abs(top)))
        fig.colorbar(cbar,cax=cbars[0,0])
        axes[0,0].set_title(f"{field['label']} - Surface")

        cbar = axes[0,1].imshow(bottom, cmap=cmap, vmin=-np.max(np.abs(bottom)), vmax=np.max(np.abs(bottom)))
        fig.colorbar(cbar,cax=cbars[0,1])
        axes[0,1].set_title(f"{field['label']} - Bottom")

    if plot:
        axes[1,0].set_xlim([times[0], times[last]])
        axes[1,0].plot(times[:i+offset+1], coefs[:i+offset+1])
        axes[1,0].set_title(f"Mean Ratio Evolution Over Frames")
        plt.pause(0.01)
if plot:
    plt.ioff()